
*** Huffmancoding is text compressing algorithm ***

# Implementation Of Huffman Coding

In [1]:
import heapq

class BinaryTreeNode:
    def __init__(self,value,freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    def __lt__(self,other):   # Less then
        return self.freq < other.freq
    
    def __eq__(self,other): # equal to
        return self.freq == other.freq
        
class HuffmanCoding:
    
    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__codes = {}
    
    def __make_frequency_dict(self,text):
        freq_dict = {}
        for char in text:
            if char not in freq_dict:
                freq_dict[char] = 0
            freq_dict[char] += 1
        return freq_dict
    
    def __buildHeap(self,freq_dict):
        for key in freq_dict:
            frequency = freq_dict[key]
            binary_tree_node = BinaryTreeNode(key,frequency)
            heapq.heappush(self.__heap,binary_tree_node)
            
    def __heapTree(self):
        while (len(self.__heap) > 1):
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            freq_sum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTreeNode(None,freq_sum)
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            heapq.heappush(self.__heap,newNode)
        return 
    
    def __buildCodesHelper(self,root,curr_bits):
        if root is None:
            return  
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            return
        self.__buildCodesHelper(self,root.left,curr_bits+"0")
        self.__buildCodesHelper(self,root.right,curr_bits+"1")
    
    def __buildCodes(self):
        root = heapq.heappop(self.__heap)
        self.___buildCodesHelper(root,"")
    
    def ____getEncodedText(self,text):
        encoded_text = ""
        for char in text:
            encoded_text += self.__codes[char]
        return encoded_text
    
    def __getPaddedEncodedText(self,encoded_text):
        padded_ammount = 8 - (len(encoded_text)%8)
        for i in range(padded_ammount):
            encoded_text += '0'
        padded_info = {"0:08b"}.format(padded_ammount)
        encoded_text = padded_info + padded_ammount
        return encoded_text
    
    def compress(self):
        # Get file form path
        
        # reak text form file
        text = "salkdjfaskd faskldfjas dfksaldkjf"
        
        # Make frequency dictionary suing the text
        freq_dict = self.__make_frequency_dict(text)
        
        # Construct the heap from the frequency dict
        self.__buildHeap(freq_dict)
        
        # Construct the binary tree from the heap
        self.__buildTree()
        
        # Construct the code from binary tree
        self.__buildCodes()
        
        # Creating encoded text using the codes
        encoded_text = self.__getEncodedText(text)
        
        # pad this encoded text
        padded_encoded_text = self.__getPaddedEncodedText(encoded_text)
        
        # Put the encoded text into the binary file
        
        # Return the binary file as output
        pass
    
    